#### Define your goals:
Determine the objective of your trading algorithm. Are you aiming for high returns, low risk, or something else? Define your metrics for success.

#### Imports

In [1]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import holoviews as hv
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import yfinance as yf

#### Data Collection:
Gather historical market data for the assets you want to trade. You can use APIs (e.g., Alpha Vantage, Yahoo Finance), paid data services, or web scraping techniques.

In [7]:
stock_symbol = 'TSM'  # Example stock symbol (you can change this)
dxy_symbol = 'DX-Y.NYB'    # Example DXY symbol (USD currency index)
start_date = '2020-01-01'
end_date = '2023-01-01'

In [8]:
tsm_df = yf.download(stock_symbol, start=start_date, end=end_date)['Close']
dxy_df = yf.download(dxy_symbol, start=start_date, end=end_date)['Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
stocks_df = pd.concat([tsm_df, dxy_df],axis=1, keys=['TSM', 'DXY'])

In [10]:
stocks_df.count()

TSM    756
DXY    756
dtype: int64

#### Data Preprocessing:
Clean and preprocess the collected data. This step may involve handling missing values, normalizing data, and feature engineering.

In [12]:
stocks_df['TSM Daily Returns'] = stocks_df['TSM'].pct_change()
stocks_df['DXY Daily Returns'] = stocks_df['DXY'].pct_change()
stocks_df.dropna(inplace=True)

stocks_df.head()

,TSM,DXY,TSM Daily Returns,DXY Daily Returns
Date,,,,
2020-01-06,57.389999,96.669998,-0.011540,-0.001755
2020-01-07,58.320000,96.980003,0.016205,0.003207
2020-01-08,58.750000,97.300003,0.007373,0.003300
2020-01-09,59.230000,97.449997,0.008170,0.001542
2020-01-10,58.860001,97.360001,-0.006247,-0.000924


#### Feature Engineering:
Create features from the raw data that can be used by your machine learning model to make predictions. These features could include technical indicators, fundamental metrics, or sentiment analysis from news articles.

In [14]:
# Creating and adding 50 moving averages of DXY & TSM to the dataframe
stocks_df['50-Day Moving Average DXY'] = stocks_df['DXY'].rolling(window=50).mean()
stocks_df['50-Day Moving Average TSM'] = stocks_df['TSM'].rolling(window=50).mean()
stocks_df.dropna(inplace=True)
stocks_df.head()

,TSM,DXY,TSM Daily Returns,DXY Daily Returns,50-Day Moving Average DXY,50-Day Moving Average TSM
Date,,,,,,
2020-05-27,51.740002,99.059998,0.018103,0.001516,100.0026,50.2882
2020-05-28,50.279999,98.379997,-0.028218,-0.006865,99.9786,50.3230
2020-05-29,50.330002,98.339996,0.000994,-0.000407,99.9222,50.4422
2020-06-01,51.029999,97.830002,0.013908,-0.005186,99.8236,50.5850
2020-06-02,52.009998,97.669998,0.019204,-0.001636,99.7206,50.7346


In [16]:
# Adding columns for cumulative returns
stocks_df['Cumulative Returns TSM'] = (1 + pd.Series(stocks_df['TSM Daily Returns'])).cumprod()
stocks_df['Cumulative Returns DXY'] = (1 + pd.Series(stocks_df['DXY Daily Returns'])).cumprod()
                                                     

In [17]:
# Slicing MA data into separate dataframe
ma_df = stocks_df[['50-Day Moving Average DXY', '50-Day Moving Average TSM']]
ma_df.head()

,50-Day Moving Average DXY,50-Day Moving Average TSM
Date,,
2020-05-27,100.0026,50.2882
2020-05-28,99.9786,50.3230
2020-05-29,99.9222,50.4422
2020-06-01,99.8236,50.5850
2020-06-02,99.7206,50.7346


In [18]:
# Create a column with RSI Data
def calculate_rsi(data, window):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Define the window size for RSI calculation
window = 14

# Calculate RSI for the 'TSM' column
stocks_df['RSI'] = calculate_rsi(stocks_df['TSM'], window)

stocks_df.head(50)

,TSM,DXY,TSM Daily Returns,DXY Daily Returns,50-Day Moving Average DXY,50-Day Moving Average TSM,Cumulative Returns TSM,Cumulative Returns DXY,RSI
Date,,,,,,,,,
2020-05-27,51.740002,99.059998,0.018103,0.001516,100.002600,50.2882,1.018103,1.001516,NaN
2020-05-28,50.279999,98.379997,-0.028218,-0.006865,99.978600,50.3230,0.989374,0.994642,NaN
2020-05-29,50.330002,98.339996,0.000994,-0.000407,99.922200,50.4422,0.990358,0.994237,NaN
2020-06-01,51.029999,97.830002,0.013908,-0.005186,99.823600,50.5850,1.004132,0.989081,NaN
2020-06-02,52.009998,97.669998,0.019204,-0.001636,99.720600,50.7346,1.023416,0.987463,NaN
2020-06-03,53.060001,97.279999,0.020188,-0.003993,99.616400,50.8906,1.044077,0.983520,NaN
2020-06-04,54.279999,96.680000,0.022993,-0.006168,99.515200,50.9988,1.068083,0.977454,NaN
2020-06-05,55.200001,96.940002,0.016949,0.002689,99.433000,51.1228,1.086187,0.980083,NaN
2020-06-08,55.570000,96.620003,0.006703,-0.003301,99.377000,51.2368,1.093467,0.976848,NaN


In [19]:
closing_price_plot = stocks_df.hvplot.line(x='Date', y='TSM', xlabel='Date', ylabel='Price', label='TSM Closing Price', color='blue', width=800, height=400)
rsi_plot = stocks_df.hvplot.line(x='Date', y='RSI', xlabel='Date', ylabel='RSI', label='RSI', color='orange', ylim=(0, 100), width=800, height=200) * hv.HLine(y=70) * hv.HLine(y=30)

closing_price_plot * rsi_plot

:Overlay
   .Curve.TSM_Closing_Price :Curve   [Date]   (TSM)
   .Curve.RSI               :Curve   [Date]   (RSI)
   .HLine.I                 :HLine   [x,y]
   .HLine.II                :HLine   [x,y]

In [12]:
# Create Signals column/df

#### Split Data:
Split your data into training, validation, and testing sets. The training set is used to train the model, the validation set is used for hyperparameter tuning, and the testing set is used to evaluate the model's performance.

In [13]:
# Assign a copy of the 50 day MA & RSI columns to a features DataFrame called X
X = # (Insert column names).shift().dropna()

# Review the DataFrame
X.head()

SyntaxError: invalid syntax (501904784.py, line 2)

In [ ]:
# Create the target set selecting the Signal column and assiging it to y
y = #Insert signal column name here

# Review the value counts
y.value_counts()

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=24)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.head()

#### Model Selection:
Choose a machine learning model suitable for your problem. Common models for trading algorithms include linear regression, decision trees, random forests, support vector machines, and neural networks.

In [ ]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Model Training:
Train your chosen model using the training data. Tune hyperparameters using the validation set to improve performance.

#### Model Evaluation:
Evaluate the trained model's performance using the testing set. Common evaluation metrics for trading algorithms include accuracy, precision, recall, and profit/loss metrics.

#### Backtesting:
Implement a backtesting framework to simulate trading based on historical data. Evaluate the model's performance on unseen data and analyze its profitability and risk-adjusted returns.

#### Optimization and Refinement:
Refine your model by iterating through steps 4-9. Experiment with different features, models, and hyperparameters to improve performance.

#### Deployment:
Once satisfied with the model's performance, deploy it in a live trading environment. Be cautious and start with small investments to mitigate risks.

#### Monitoring and Maintenance:
Continuously monitor the model's performance in real-time trading and update it as needed. Markets change, so your model may need adjustments over time